In [31]:
from sklearn.datasets import fetch_20newsgroups
from sklearn.cluster import KMeans as sk_kmeans
from nltk.cluster.kmeans import KMeansClusterer as nl_kmeans
from sklearn.cluster import AgglomerativeClustering
from nltk.cluster.util import cosine_distance
import numpy as np
import pandas as pd
from sklearn import metrics

import torch
from transformers import BertTokenizer, BertModel

In [32]:
file = open('20docs3UnDif.txt', 'r')
corpus = [line.strip() for line in file]
file.close()

In [33]:
#'comp.graphics', 'rec.autos', 'sci.med',  'talk.politics.mideast'
categories = ['talk.politics.guns', 'talk.politics.mideast',  'talk.politics.misc']
newsgroups = fetch_20newsgroups(subset='all', remove=('headers', 'footers', 'quotes'), categories=categories)
Y = newsgroups.target

Base Cased

In [34]:
model_name = 'bert-base-cased'
tokenizer = BertTokenizer.from_pretrained(model_name)
tokens = [tokenizer(i, return_tensors='pt', padding=True, truncation=True) for i in corpus]

In [35]:
model = BertModel.from_pretrained(model_name)
X = np.array([[]])
for i in tokens:
    with torch.no_grad():
        outputs = model(**i)
        embedding = outputs.last_hidden_state[:, 0, :]
    X = np.append(X,embedding)

Some weights of the model checkpoint at bert-base-cased were not used when initializing BertModel: ['cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [36]:
X = np.reshape(X, (2625,768))
iter = 30
a_rand = np.zeros((2,iter))
v_measure = np.zeros((2,iter))
mutual = np.zeros((2,iter))
fowlkes = np.zeros((2,iter))

In [37]:
for i in range(iter):
    eukl_pred = sk_kmeans(n_clusters=3, init='k-means++', n_init='auto').fit(X)
    eucl_pred = eukl_pred.labels_
    a_rand[0][i] = metrics.rand_score(Y, eucl_pred)
    v_measure[0][i] = metrics.v_measure_score(Y,eucl_pred)
    mutual[0][i] = metrics.adjusted_mutual_info_score(Y,eucl_pred)
    fowlkes[0][i] = metrics.fowlkes_mallows_score(Y, eucl_pred)

    nl_clusterer = nl_kmeans(3, distance=cosine_distance, avoid_empty_clusters=True)
    cos_pred = nl_clusterer.cluster(X, assign_clusters=True)
    cos_pred = np.array(cos_pred)
    a_rand[1][i] = metrics.rand_score(Y, cos_pred)
    v_measure[1][i] = metrics.v_measure_score(Y, cos_pred)
    mutual[1][i] = metrics.adjusted_mutual_info_score(Y, cos_pred)
    fowlkes[1][i] = metrics.fowlkes_mallows_score(Y, cos_pred)

In [38]:
evcl = pd.DataFrame({'a_rand': [np.min(a_rand[0]), np.mean(a_rand[0]), np.max(a_rand[0])],
        'v_measure': [np.min(v_measure[0]), np.mean(v_measure[0]), np.max(v_measure[0])],
        'mutual': [np.min(mutual[0]), np.mean(mutual[0]), np.max(mutual[0])],
        'fowlkes': [np.min(fowlkes[0]), np.mean(fowlkes[0]), np.max(fowlkes[0])]},
        index=['min', 'avrg', 'max'])
evcl

,a_rand,v_measure,mutual,fowlkes
min,0.490419,0.016366,0.015436,0.305832
avrg,0.561450,0.016678,0.015674,0.344345
max,0.585027,0.021218,0.019868,0.441864


In [39]:
cos = pd.DataFrame({'a_rand': [np.min(a_rand[1]), np.mean(a_rand[1]), np.max(a_rand[1])],
        'v_measure': [np.min(v_measure[1]), np.mean(v_measure[1]), np.max(v_measure[1])],
        'mutual': [np.min(mutual[1]), np.mean(mutual[1]), np.max(mutual[1])],
        'fowlkes': [np.min(fowlkes[1]), np.mean(fowlkes[1]), np.max(fowlkes[1])]},
        index=['min', 'avrg', 'max'])
cos

,a_rand,v_measure,mutual,fowlkes
min,0.491580,0.016680,0.015744,0.304171
avrg,0.562035,0.017287,0.016303,0.344057
max,0.586555,0.022119,0.021291,0.442118


In [40]:
a_rand_h = np.zeros((2,4))
v_measure_h = np.zeros((2,4))
mutual_h = np.zeros((2,4))
fowlkes_h = np.zeros((2,4))

In [41]:
linkage = ['ward', 'complete', 'average', 'single']
for i,link in enumerate(linkage):
    hierachical = AgglomerativeClustering(n_clusters=3, linkage=link).fit(X)
    eucl_pred = hierachical.labels_
    a_rand_h[0][i] = metrics.rand_score(Y, eucl_pred)
    v_measure_h[0][i] = metrics.adjusted_rand_score(Y, eucl_pred)
    mutual_h[0][i] = metrics.homogeneity_score(Y, eucl_pred)
    fowlkes_h[0][i] = metrics.completeness_score(Y, eucl_pred)

In [42]:
linkage = ['complete', 'average', 'single']
for i,link in enumerate(linkage):
    hierachical = AgglomerativeClustering(n_clusters=3, linkage=link, metric='cosine').fit(X)
    eucl_pred = hierachical.labels_
    a_rand_h[1][i] = metrics.rand_score(Y, eucl_pred)
    v_measure_h[1][i] = metrics.adjusted_rand_score(Y, eucl_pred)
    mutual_h[1][i] = metrics.homogeneity_score(Y, eucl_pred)
    fowlkes_h[1][i] = metrics.completeness_score(Y, eucl_pred)

In [43]:
eucl_h = pd.DataFrame({'a_rand': a_rand_h[0],
        'v_measure': v_measure_h[0],
        'mutual': mutual_h[0],
        'fowlkes': fowlkes_h[0]},
        index=['ward', 'complete', 'average', 'single'])
eucl_h

,a_rand,v_measure,mutual,fowlkes
ward,0.544066,0.020145,0.020342,0.020593
complete,0.476747,0.011633,0.014454,0.024835
average,0.336260,0.000280,0.001275,0.137658
single,0.336260,0.000280,0.001275,0.137658


In [44]:
cos_h = pd.DataFrame({'a_rand': a_rand_h[1],
        'v_measure': v_measure_h[1],
        'mutual': mutual_h[1],
        'fowlkes': fowlkes_h[1]},
        index=['complete', 'average', 'single', 'ward'])
cos_h

,a_rand,v_measure,mutual,fowlkes
complete,0.447130,0.006023,0.013218,0.024476
average,0.336181,0.000160,0.001218,0.131524
single,0.336163,0.000134,0.001206,0.130305
ward,0.000000,0.000000,0.000000,0.000000
